# Real life data

## Preprocessing

In [1]:
%load_ext autoreload 
%autoreload 2

import re
import sys
import hashlib

import pandas as pd
import numpy as np
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper
import tasks.preprocessing

In [2]:
df = pd.read_pickle("../data/attitudes_embedded.csv")
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]

In [3]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(np.int64(41), np.int64(41), np.int64(39))

In [4]:
def process_age_list(x, bins):
    if not isinstance(x, (list, tuple)):
        return None
    if any(pd.isna(age) for age in x):
        return None
    try:
        int_ages = [int(age) for age in x]
        return pd.cut(int_ages, bins=bins, include_lowest=True)
    except Exception:
        return None


all_ages = [
    age
    for sublist in df["annotatorAge"]
    if isinstance(sublist, (list, tuple))
    for age in sublist
    if pd.notna(age)
]
all_ages = list(map(int, all_ages))
num_bins = 4
bin_edges = [0, 20, 40, 60, 80]

df["annotatorAge"] = df["annotatorAge"].apply(
    lambda x: process_age_list(x, bin_edges)
)
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)

df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)
df = df.dropna()
df["random"] = tasks.preprocessing.get_rand_col(df, "annotatorAge")

## Results

We hypothesize that all of the comments belong in the same discussion.

In [5]:
sdb_columns = ["annotatorAge", "annotatorRace", "annotatorGender"]
res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    value_col="racism",
    comment_key_col="tweet",
)
res

Evaluating SDB dimensions:   0%|          | 0/3 [00:00<?, ?it/s]

/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0         1
sdb_column                                        
annotatorAge    (-0.001, 20.0]  0.000000  1.000000
                (20.0, 40.0]    0.031833  1.000000
                (40.0, 60.0]   -0.056498  1.000000
                (60.0, 80.0]   -0.905972  1.000000
annotatorGender man             0.145492  0.817204
                nonBinary       0.000000  1.000000
                woman           0.050714  0.817204
annotatorRace   black           0.517632  1.000000
                hisp            0.000000  1.000000
                middleEastern   0.000000       NaN
                other           0.000000  1.000000
                white          -0.250250  1.000000

In [ ]:
tasks.run_helper.run_result(
    df,
    sdb_column="random",
    value_col="racism",
    comment_key_col="tweet",
)